In [298]:
import pandas as pd
import numpy as np
from scipy import stats
import os

In [299]:

def import_files():
    gender = pd.read_csv('files/gender.csv')
    patients = pd.read_csv('files/patients.csv', infer_datetime_format=True)
    medications = pd.read_csv('files/medications.csv')
    immunizations = pd.read_csv('files/immunizations.csv')
    codes = pd.read_csv('files/codes.csv')
    return gender, patients, medications, immunizations, codes

def get_full_df():
    gender, patients, medications, immunizations, codes = import_files()

    person = make_person(patients, gender)
    drug_exposure = make_drug_exposure(medications,immunizations, codes)

    return pd.merge(person, drug_exposure,left_on=['source', 'Id'], right_on=['source', 'PATIENT'], how='outer')

def make_drug_exposure(medications,immunizations, codes ):
    medications = to_string(medications)
    immunizations = to_string(medications)

    medications_inc = pd.merge(medications,codes,left_on='CODE', right_on='source')
    immunizations_inc = pd.merge(immunizations,codes,left_on='CODE', right_on='source')
    medications_inc.rename(columns={"DESCRIPTION": "DESCRIPTION_MEDS",
                                    'BASE_COST':"BASE_COST_MEDS",
                                    'TOTALCOST':'TOTALCOST_Meds'}, inplace=True)
    immunizations_inc.rename(columns={"DESCRIPTION": "DESCRIPTION_IMMU",
                                      'BASE_COST':"BASE_COST_IMMU",
                                      'TOTALCOST':'TOTALCOST_IMMU'
                                      }, inplace=True)
    drug_exposure = pd.merge(medications_inc, immunizations_inc, on=['PATIENT', 'ENCOUNTER', 'CODE', 'source', 'domain', 'target'], how='outer')
    return drug_exposure

def to_string(df):
   df['CODE'] = df['CODE'].astype(str)
   return df

# Making 'person'
def make_person(patients, gender):
    person = patients.merge(gender, left_on='GENDER', right_on='source')
    person.rename(columns={"target": "gender_code"} , inplace=True)
    return person

def fill_missing(df):
    df.fillna(-1, inplace=True)
    return df

def cols_toupper(df):
    return df.rename(str.upper, axis='columns')

def convert_dt(df):
    df[['START', 'STOP', 'BIRTHDATE', 'DEATHDATE']] = pd.to_datetime(df[['START', 'STOP', 'BIRTHDATE', 'DEATHDATE']]).date
    return df.rename(str.upper, axis='columns')

# Probably still have to do EDA to select the columns instead of just doing it for all cols
def filter_outliers(df):
    return df[(np.abs(stats.zscore(df[0])) < 3)]

def clean_data(df):
    df = cols_toupper(df)

    df = fill_missing(df)

    df = convert_dt(df)

    df = filter_outliers(df)

    return df

def do_eda():
    full_df = get_full_df()
    output = clean_data(full_df)
    return output


# Some basic EDA
print(gender.head())
print(patients.head())
print(medications.head())
print(immunizations.head())
print(codes.head())

print(gender.describe())
print(patients.describe())
print(medications.describe())
print(immunizations.describe())
print(codes.describe())

print(gender.info())
print(patients.info())
print(medications.info())
print(immunizations.info())
print(codes.info())

In [300]:
full_df = get_full_df()
full_df.isna().sum()


Id                     583
BIRTHDATE              583
DEATHDATE              608
SSN                    583
DRIVERS                586
PASSPORT               587
PREFIX                 586
FIRST                  583
LAST                   583
SUFFIX                 608
MAIDEN                 606
MARITAL                589
RACE                   583
ETHNICITY              583
GENDER                 583
BIRTHPLACE             583
ADDRESS                583
CITY                   583
STATE                  583
COUNTY                 583
ZIP                    593
LAT                    583
LON                    583
HEALTHCARE_EXPENSES    583
HEALTHCARE_COVERAGE    583
source                   0
gender_code            583
START_x                 25
STOP_x                  60
PATIENT                 25
PAYER_x                 25
ENCOUNTER               25
CODE                    25
DESCRIPTION_MEDS        25
BASE_COST_MEDS          25
PAYER_COVERAGE_x        25
DISPENSES_x             25
T

In [301]:
if __name__ == '__main__':
    print(do_eda())

KeyError: "['START', 'STOP'] not in index"